In [1]:
import openai
import json
import os

from transformers import XLMRobertaTokenizer
from transformers import pipeline

2023-11-23 22:10:34.911118: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-23 22:10:34.913643: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-23 22:10:34.951643: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 22:10:34.951671: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 22:10:34.951702: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [6]:
'''
read config file
'''
config_file='config.json' #expected that you keep it in the current directory
with open(os.path.join('./', config_file), 'r') as json_file:
    config = json.load(json_file)

In [7]:
config

{'model_name': 'xlm-roberta-base',
 'file': 'Train_Numerical_Reasoning.json',
 'path': '../NumEval - Task 3/'}

In [8]:
path=config['path']
file=config['file']

In [9]:
with open(os.path.join(path, file), 'r') as json_file:
    data = json.load(json_file)

In [10]:
model_name=config['model_name']
classifier = pipeline("fill-mask", model=model_name)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
MASK_TOKEN = tokenizer.mask_token

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def get_input_data(inp_id):
    
    X = (data[inp_id]['news'] + data[inp_id]['masked headline']).replace('____', MASK_TOKEN)
    Y = (data[inp_id]['news'] + data[inp_id]['masked headline']).replace('____',  data[inp_id]['ans'])
    return X, Y

def get_prompt(data):
    prompt = "Read the news article and fill in the mask in the last sentence. The answer is present in the text and it is a numeric value. " + data    
    return prompt

In [13]:
'''
Example 1
'''
X, Y = get_input_data(2)
prompt = get_prompt(X)
prompt

"Read the news article and fill in the mask in the last sentence. The answer is present in the text and it is a numeric value. (Mar 6, 2016  10:50 AM) Nancy Reagan, the helpmate, backstage adviser, and fierce protector of Ronald Reagan in his journey from actor to president—and finally during his 10-year battle with Alzheimer's disease—died Sunday at the age of 94, reports the AP, via CBS News. The cause was congestive heart failure, notes ABC News. In addition to her famous campaign against drugs, the one-time actress promoted several causes while she was in the White House and even in the years after. She was a passionate advocate for lifting restrictions on stem cell research and promoting better treatment of America's veterans. But above all, Nancy Reagan was a fiercely devoted wife.  My life began with Ronnie,  she told Vanity Fair magazine in 1998. The first lady's public life had its share of controversy but also earned the respect of the nation, making Nancy Reagan one of Ameri

In [14]:
Y

"(Mar 6, 2016  10:50 AM) Nancy Reagan, the helpmate, backstage adviser, and fierce protector of Ronald Reagan in his journey from actor to president—and finally during his 10-year battle with Alzheimer's disease—died Sunday at the age of 94, reports the AP, via CBS News. The cause was congestive heart failure, notes ABC News. In addition to her famous campaign against drugs, the one-time actress promoted several causes while she was in the White House and even in the years after. She was a passionate advocate for lifting restrictions on stem cell research and promoting better treatment of America's veterans. But above all, Nancy Reagan was a fiercely devoted wife.  My life began with Ronnie,  she told Vanity Fair magazine in 1998. The first lady's public life had its share of controversy but also earned the respect of the nation, making Nancy Reagan one of America's most admired women in the 1980s and beyond. Anne Frances  Nancy  Robbins was born on July 6, 1921 in New York City to Ken

In [15]:
'''
Model predictions
'''
classifier(prompt)

[{'score': 0.9198079705238342,
  'token': 16064,
  'token_str': '94',
  'sequence': "Read the news article and fill in the mask in the last sentence. The answer is present in the text and it is a numeric value. (Mar 6, 2016 10:50 AM) Nancy Reagan, the helpmate, backstage adviser, and fierce protector of Ronald Reagan in his journey from actor to president—and finally during his 10-year battle with Alzheimer's disease—died Sunday at the age of 94, reports the AP, via CBS News. The cause was congestive heart failure, notes ABC News. In addition to her famous campaign against drugs, the one-time actress promoted several causes while she was in the White House and even in the years after. She was a passionate advocate for lifting restrictions on stem cell research and promoting better treatment of America's veterans. But above all, Nancy Reagan was a fiercely devoted wife. My life began with Ronnie, she told Vanity Fair magazine in 1998. The first lady's public life had its share of contro

In [16]:
'''
Example 2
'''
X, Y = get_input_data(10)
prompt = get_prompt(X)
prompt

"Read the news article and fill in the mask in the last sentence. The answer is present in the text and it is a numeric value. (Sep 15, 2015  4:39 PM CDT) Three people died and others were injured today after a floatplane belonging to a fishing lodge crashed with 10 people on board in southwest Alaska, authorities said. Alaska State Troopers said the three dead were from outside the state. The injured passengers were being flown to Anchorage hospitals following the crash near the small community of Iliamna, 175 miles southwest of Anchorage. It's unclear how many people were hurt. Local residents said the plane belongs to the Rainbow King Lodge. Rescue personnel initially took injured passengers to the clinic in Iliamna. The plane crashed on takeoff at Eastwind Lake, 1 mile north of the community, according to Federal Aviation Administration spokesman Allen Kenitzer. It was a De Havilland DHC-3 Turbine Otter on floats, he said. The cause of the crash is not yet known. National Transport

In [17]:
Y

"(Sep 15, 2015  4:39 PM CDT) Three people died and others were injured today after a floatplane belonging to a fishing lodge crashed with 10 people on board in southwest Alaska, authorities said. Alaska State Troopers said the three dead were from outside the state. The injured passengers were being flown to Anchorage hospitals following the crash near the small community of Iliamna, 175 miles southwest of Anchorage. It's unclear how many people were hurt. Local residents said the plane belongs to the Rainbow King Lodge. Rescue personnel initially took injured passengers to the clinic in Iliamna. The plane crashed on takeoff at Eastwind Lake, 1 mile north of the community, according to Federal Aviation Administration spokesman Allen Kenitzer. It was a De Havilland DHC-3 Turbine Otter on floats, he said. The cause of the crash is not yet known. National Transportation Safety Board Alaska Chief Clint Johnson said the plane came to rest in some trees. Two NTSB investigators were heading t

In [18]:
'''
Model predictions
'''
classifier(prompt)

[{'score': 0.2777342200279236,
  'token': 2,
  'token_str': '</s>',
  'sequence': "Read the news article and fill in the mask in the last sentence. The answer is present in the text and it is a numeric value. (Sep 15, 2015 4:39 PM CDT) Three people died and others were injured today after a floatplane belonging to a fishing lodge crashed with 10 people on board in southwest Alaska, authorities said. Alaska State Troopers said the three dead were from outside the state. The injured passengers were being flown to Anchorage hospitals following the crash near the small community of Iliamna, 175 miles southwest of Anchorage. It's unclear how many people were hurt. Local residents said the plane belongs to the Rainbow King Lodge. Rescue personnel initially took injured passengers to the clinic in Iliamna. The plane crashed on takeoff at Eastwind Lake, 1 mile north of the community, according to Federal Aviation Administration spokesman Allen Kenitzer. It was a De Havilland DHC-3 Turbine Otte